In [1]:
import requests
import pandas as pd 
import numpy as np
import os
from os import listdir
from os.path import isfile, join
import pathlib
import urllib
import urllib.request
import zipfile
import io

In [9]:
def download_file(url, local_filename):
    # NOTE the stream=True parameter below
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=16384): 
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)
                    # f.flush()
    return local_filename


def buildUrl(year, month):
    return "https://s3.amazonaws.com/capitalbikeshare-data/" + str(year) + "{0:0=2d}".format(month) + "-capitalbikeshare-tripdata.zip"

def downloadZipFile(url):
    r = requests.get(url)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    ourPath =  str(pathlib.Path().absolute()) + "/data/"
    print(ourPath)
    z.extractall(ourPath)

def downloadAllZipFiles():
    year = 2019
    month = 2
    while month < 13:
        url = buildUrl(year,month)
        downloadZipFile(url)

        month += 1
    
def makeFrame():
    ourPath = str(pathlib.Path().absolute()) + "/data/"

    onlyFiles = [f for f in listdir(ourPath) if isfile(join(ourPath, f))]

    dfs = []
    for fileName in onlyFiles:
        if ("-capitalbikeshare-tripdata" in fileName):
            print(fileName)
            fileName = str(ourPath + fileName)
            mylist = []

            for chunk in  pd.read_csv(fileName, encoding = "utf-8", chunksize=20000):
                mylist.append(chunk)
                
            os.remove(fileName)

            big_data = pd.concat(mylist, axis= 0)
            del(mylist)
            dfs.append(big_data)

    # Concatenate all data into one DataFrame
    big_frame = pd.concat(dfs, ignore_index=True)
    return big_frame
    

In [11]:
# get all open data dc data for the project

url = "https://opendata.arcgis.com/datasets/70392a096a8e431381f1f692aaa06afd_24.csv"
fileName = download_file(url, "data/source data/accidents.csv")

url = "https://opendata.dc.gov/datasets/294e062cdf2c48d5b9cbc374d9709bc0_2.csv"
fileName = download_file(url,"data/source data/streets.csv")

url = "https://opendata.dc.gov/datasets/294e062cdf2c48d5b9cbc374d9709bc0_2.kml"
fileName = download_file(url,"data/source data/streets.kml")

url = "https://opendata.arcgis.com/datasets/f21813094ed24fd5ba66a07b61a1acee_46.csv"
fileName = download_file(url,"data/source data/streetSegs.csv")

url = "https://opendata.arcgis.com/datasets/a1f7acf65795451d89f0a38565a975b3_5.csv"
fileName = download_file(url,"data/source data/bikeShareLocations.csv")

In [10]:
# get bike share data
downloadAllZipFiles()
big_frame = makeFrame()
big_frame.to_csv("data/source data/bikeShareTrips.csv")

/User/jgluck/Documents/GitHub/dc-bike-analysis/data/
/User/jgluck/Documents/GitHub/dc-bike-analysis/data/
/User/jgluck/Documents/GitHub/dc-bike-analysis/data/
/User/jgluck/Documents/GitHub/dc-bike-analysis/data/
/User/jgluck/Documents/GitHub/dc-bike-analysis/data/
/User/jgluck/Documents/GitHub/dc-bike-analysis/data/
/User/jgluck/Documents/GitHub/dc-bike-analysis/data/
/User/jgluck/Documents/GitHub/dc-bike-analysis/data/
/User/jgluck/Documents/GitHub/dc-bike-analysis/data/
/User/jgluck/Documents/GitHub/dc-bike-analysis/data/
/User/jgluck/Documents/GitHub/dc-bike-analysis/data/
201910-capitalbikeshare-tripdata.csv
201909-capitalbikeshare-tripdata.csv
201906-capitalbikeshare-tripdata.csv
201911-capitalbikeshare-tripdata.csv
201904-capitalbikeshare-tripdata.csv
201908-capitalbikeshare-tripdata
201903-capitalbikeshare-tripdata.csv
201912-capitalbikeshare-tripdata.csv
201907-capitalbikeshare-tripdata
201905-capitalbikeshare-tripdata.csv
201902-capitalbikeshare-tripdata.csv
